### MeteOcean wave climate and extremes statistics in the Mediterranean Sea: hindcast and multi-model ensemble of GCM-RCMs projections by 2100
#### A. Lira-Loarca & G. Besio



This notebook loads and plots the indices for hindcast and future projections. The dataset contains statistics for significant wave height, $H_s$, mean wave period, $T_m$, peak wave period, $T_p$, and mean wave direction, $\theta_m$, for Hindcast (1979-2005) and a multi-model ensemble of 17 EURO-CORDEX GCM-RCMs projections for the following periods: 
- Baseline/Hindcast (1979-2005)
- Mid-century (2034-2060) for RCP 8.5
- End-of-century (2074-2100) for RCP 8.5.

The user needs to select the variable <code>var</code> and the case <code>case</code> to analyze. 

Additionally,for `var = 'hs'`, the user needs to define `bias_adj = True` or `bias_adj = False`.

The different options are:

For `var = 'hs'`:
- `case = `
    - `'monthlymax'`| `'seasonalmax'`
    - `'monthlymean'` | `'seasonalmean'`
    - `'monthlyquantiles'` | `'seasonalquantiles'`
    - `meanseastates_over_p90__monthlymean` | `meanseastates_over_p90__seasonalmean`
    - `meanseastates_over_p95__monthlymean` | `meanseastates_over_p95__seasonalmean`
    - `nseastates_over_p90__monthlymean` | `nseastates_over_p90__seasonalmean`
    - `nseastates_over_p95__monthlymean` | `nseastates_over_p95__seasonalmean`
    - `nseastates_over_p90__monthlymean_perc` | `nseastates_over_p90__seasonalmean_perc`
    - `nseastates_over_p95__monthlymean_perc` | `nseastates_over_p95__seasonalmean_perc`
    - `meanseastates_over_ss3-1p25__monthlymean` | `meanseastates_over_ss3-1p25__seasonalmean`
    - `meanseastates_over_ss4-2p5__monthlymean` | `meanseastates_over_ss4-2p5__seasonalmean`
    - `meanseastates_over_ss5-4__monthlymean` | `meanseastates_over_ss5-4__seasonalmean`
    - `nseastates_over_ss3-1p25__monthlymean` | `nseastates_over_ss3-1p25__seasonalmean`
    - `nseastates_over_ss4-2p5__monthlymean` | `nseastates_over_ss4-2p5__seasonalmean`
    - `nseastates_over_ss5-4__monthlymean` | `nseastates_over_ss5-4__seasonalmean` 
    - `nseastates_over_ss3-1p25__monthlymean_perc` |  `nseastates_over_ss3-1p25__seasonalmean_perc`
    - `nseastates_over_ss4-2p5__monthlymean_perc` | `nseastates_over_ss4-2p5__seasonalmean_perc`
    - `nseastates_over_ss5-4__monthlymean_perc` |  `nseastates_over_ss5-4__seasonalmean_perc`
    - `ndays_dmax_over_p90_2consecdays__monthlymean` | `ndays_dmax_over_p90_2consecdays__seasonalmean`
    - `ndays_dmax_over_p95_2consecdays__monthlymean` | `ndays_dmax_over_p95_2consecdays__seasonalmean`
    - `ndays_dmax_over_p90_2consecdays__monthlymean_perc` | `ndays_dmax_over_p90_2consecdays__seasonalmean_perc`
    - `ndays_dmax_over_p95_2consecdays__monthlymean_perc` | `ndays_dmax_over_p95_2consecdays__seasonalmean_perc`

For <code>var = 'tm'</code> or `var = 'tp'`
- <code>case = </code>
    - <code>'monthlymax'</code> | <code>'seasonalmax'</code>
    - <code>'monthlymean'</code> | <code>'seasonalmean'</code>
    - <code>'monthlyquantiles'</code> | <code>'seasonalquantiles'</code>

For <code>var = 'dirm'</code>
- <code>case = </code>
    - <code>'monthlymean'</code> | <code>'seasonalmean'</code>
    - <code>'monthlystd'</code> | <code>'seasonalstd'</code>




In [1]:
var = 'tm'
case = 'seasonalquantiles'
bias_adj = False  # only for hs

In [2]:
import fsspec 
import xarray as xr
import hvplot.xarray
import pandas as pd

In [3]:
fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True,
                            client_kwargs={'endpoint_url': 'https://usgs.osn.mghpcc.org'})

if var == 'hs' and bias_adj:
    data_ba = 'ba_eqm_month'
else:
    data_ba = 'raw' 

In [4]:
# Hindcast
data_url = f's3://genoatest/aloarca/wave_dataset/hindcast_{var}_1979_2005__{case}.nc'
ds_hind = xr.open_dataset(fs_read.open(data_url), engine='h5netcdf')
ds_hind[var].hvplot(x='longitude', y='latitude', rasterize=True)

ERROR 1: PROJ: proj_create_from_database: Open of /opt/miniforge3/envs/pangeo/share/proj failed
/opt/miniforge3/envs/pangeo/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)
/opt/miniforge3/envs/pangeo/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
/opt/miniforge3/envs/pangeo/lib/python3.10/site-packages/holoviews/core/util.py:1175: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return pd.unique(values)


:DynamicMap   [season,quantile]
   :Image   [longitude,latitude]   (tm)

In [ ]:
# GCM-RCMs
periods = ['rcp85_mid', '2034_2060'], ['rcp85_end', '2074_2100']
list_fn_rcms_per, list_case_rcms_per = list(), list()
for period in periods:
    list_fn_rcms, list_case_rcms = list(), list()
    data_url = f's3://genoatest/aloarca/wave_dataset/*_{data_ba}_{period[0]}_{var}_{period[1]}__{case}.nc'
    for fn in fs_read.glob(data_url):
        ds_rcm = xr.open_dataset(fs_read.open(fn), engine='h5netcdf')
        ds_rcm = ds_rcm.reindex(longitude=ds_hind.longitude, latitude=ds_hind.latitude, method='nearest')
        list_fn_rcms.append(ds_rcm)
        list_case_rcms.append(fn.split('/')[-1].split('_')[0])
    ds_rcms = xr.concat(list_fn_rcms, pd.Index(list_case_rcms, name='case'))
    list_fn_rcms_per.append(ds_rcms)
    list_case_rcms_per.append(f'{period[0]}-{period[1]}')
ds_rcms = xr.concat(list_fn_rcms_per, pd.Index(list_case_rcms_per, name='period'))

list_fn, list_case = list(), list()
list_fn.append(ds_rcms.mean('case'))
list_case.append('ensemble-mean')

list_fn.append(ds_rcms.mean('case')-ds_hind)
list_case.append('changes-em-hindcast')

ds = xr.concat(list_fn, pd.Index(list_case, name='case'))
ds = xr.merge([ds, ds_rcms])
ds[var].hvplot(x='longitude', y='latitude', rasterize=True)